<a href="https://colab.research.google.com/github/alibelhrak/RNN_Medical_Chatbot/blob/main/Second_Edition_Of__RNN_Healthcare_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Dataset

In [1]:
pip install opendatasets

In [2]:
import opendatasets as od
ds = od.download('https://www.kaggle.com/datasets/yousefsaeedian/ai-medical-chatbot/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alibelhrak 
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/yousefsaeedian/ai-medical-chatbot


In [3]:
import pandas as pd
df = pd.read_csv('/content/ai-medical-chatbot/ai-medical-chatbot.csv')
df


,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...
...,...,...,...
256911,Why is hair fall increasing while using Bontre...,I am suffering from excessive hairfall. My doc...,"Hello Dear Thanks for writing to us, we are he..."
256912,Why was I asked to discontinue Androanagen whi...,"Hi Doctor, I have been having severe hair fall...","hello, hair4u is combination of minoxid..."
256913,Can Mintop 5% Lotion be used by women for seve...,Hi..i hav sever hair loss problem so consulted...,HI I have evaluated your query thoroughly you...
256914,Is Minoxin 5% lotion advisable instead of Foli...,"Hi, i am 25 year old girl, i am having massive...",Hello and Welcome to ‘Ask A Doctor’ service.I ...


# ***I can't take the hole dataset cause colab is not capable to deal with so i only took 10000 row and i made small chunks to make it easier for the model fitting ***

In [4]:
import pandas as pd
df = pd.read_csv("/content/ai-medical-chatbot/ai-medical-chatbot.csv", nrows=10000)


In [5]:
small_data = []

for i, chunk in enumerate(pd.read_csv("/content/ai-medical-chatbot/ai-medical-chatbot.csv", chunksize=5000)):
    small_data.append(chunk)
    if i == 1:
        break

df_small = pd.concat(small_data)
print(df_small.shape)


(10000, 3)


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
Description = df_small['Description'].astype(str)
patient = df_small['Patient'].astype(str)
outputs = df_small['Doctor'].astype(str)

# Preprocessing Data

In [8]:
pip install nltk

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
from collections.abc import Sequence
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


# Hne just bech tne77i l rows zeydin
df_small = df_small.dropna()
df_small = df_small.drop_duplicates()


# hne bech t7ot description m3a l questions
inputs = (Description + " " + patient.astype(str))


# to tokenize l inputs wel outputs
tokenizer_input = Tokenizer()
tokenizer_output = Tokenizer()


tokenizer_input.fit_on_texts(inputs)
tokenizer_output.fit_on_texts(outputs)


# hne bech t7ethom fi des sequences
input_sequences = tokenizer_input.texts_to_sequences(inputs)
output_sequences = tokenizer_output.texts_to_sequences(outputs)


# heth l max ta3 les sequences
max_input_len = 50
max_output_len = 50


# hne bech t7ot sequences fi padds (ekher haja hiya l padds ta3 sequences )

input_padded = pad_sequences(input_sequences , maxlen = max_input_len , padding='post')
output_padded = pad_sequences(output_sequences ,maxlen  = max_output_len , padding='post')

In [11]:
input_padded

array([[  15,   22,   87, ...,    0,    0,    0],
       [ 121,  498,    4, ..., 2845,    2, 8204],
       [ 950,  493,  606, ..., 1474, 1290, 1365],
       ...,
       [1077,  500,  913, ...,   23,    2,  409],
       [   7,   96, 1973, ...,  524,   30,   74],
       [  15,    7,   63, ...,    0,    0,    0]], dtype=int32)

In [12]:
output_padded

array([[  32,   16,   19, ...,    0,    0,    0],
       [ 350,   10, 1244, ...,   36,  262,   49],
       [  19,  479,    2, ..., 8758,  661,  698],
       ...,
       [1173,  569, 1509, ...,    6,  268,   28],
       [  32,    8,  934, ...,    0,    0,    0],
       [  35,  639,    3, ...,    0,    0,    0]], dtype=int32)

# Splitting the Data

In [13]:
from sklearn.model_selection import train_test_split
# les X bennesba laya lehne houma l inputs padds , w les Y bennesba laya houma l outputs padds
X_train , X_test , y_train , y_test = train_test_split(input_padded , output_padded , test_size = 0.2)

# Building The Model

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , LSTM , Dense , Embedding


# 3dad l kelmet elli bech n7othom (elli houma tokenizes words fi word_index w dima nzid 1 khater python kima ta3ref ya si ali .. )

vocab_size_input = len(tokenizer_input.word_index) + 1
vocab_size_output = len(tokenizer_output.word_index) +1

# hetha l embedding win bech yhawel l kelma l vector fih 100 dim
embedding_dim  = 100

# 3dad nodes fi LSTM
latent_dim = 256


# Toul akbar 7aja fil inputs
max_input_len = input_padded.shape[1]
# Nafs chay ll ouputs
max_output_len = output_padded.shape[1]

# hethi l partie ta3 l encoder

# bech yekho l inputs les phrases
encoder_inputs = Input(shape=(max_input_len,))

# l embedding ta3 l input y7awel lkelma l vector
enc_emb = Embedding(vocab_size_input, embedding_dim)(encoder_inputs)

# l encoder ta3 couche l LSTM ya9ra mot par mot bel return state
encoder_lstm = LSTM(latent_dim, return_state=True)


# hethom l hidden  layers cellulet ba3d l 9raya
_, state_h, state_c = encoder_lstm(enc_emb)


# ekher mar7la t7ot l hidden state w cellulet
encoder_states = [state_h, state_c]



# Hetheya l partie ta3 decoder (win l model yebda yet3allem bech ykawen les phrases ) => l partie hethi tkhos l outputs

max_decoder_seq_length = max_output_len - 1

decoder_inputs = Input(shape=(max_decoder_seq_length,))
dec_emb = Embedding(vocab_size_output, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

# lehne bech yebda ykawen les phrases en se basent 3al enc_emb
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Hetheya l model bkolo
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 49)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 100)   │  1,849,200 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 49, 100)   │  1,778,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    365,568 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 49, 256), │    365,568 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm[0][1],       │
│                     │ (None, 256)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 49, 17780) │  4,569,460 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,927,796 (34.06 MB)

 Trainable params: 8,927,796 (34.06 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input


encoder_model_inf = Model(encoder_inputs, encoder_states)


decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_single_input = Input(shape=(1,))



dec_emb_layer = Embedding(vocab_size_output, embedding_dim)
dec_emb_inf = dec_emb_layer(decoder_single_input)

decoder_lstm_layer = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm_layer(
    dec_emb_inf, initial_state=decoder_states_inputs
)
decoder_states = [state_h_inf, state_c_inf]

decoder_dense_layer = Dense(vocab_size_output, activation='softmax')
decoder_outputs_inf = decoder_dense_layer(decoder_outputs_inf)

decoder_model_inf = Model(
    [decoder_single_input] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states
)


In [ ]:
import numpy as np

model.fit([input_padded, output_padded[:, :-1]],
          output_padded[:, 1:, np.newaxis],
          batch_size=10,
          epochs=5,
          validation_split=0.2)


# Saving Training file , token file

In [ ]:
import pickle

model.save("medical_chatbot_model.h5")

with open("tokenizer_input.pkl", "wb") as f:
    pickle.dump(tokenizer_input, f)

with open("tokenizer_output.pkl", "wb") as f:
    pickle.dump(tokenizer_output, f)


# Loading The Model

In [15]:
from tensorflow.keras.models import load_model
model1 = load_model('/content/medical_chatbot_model.h5')

In [22]:
import pickle
with open("/content/tokenizer_input.pkl", "rb") as f:
    tokenizer_input = pickle.load(f)

with open("/content/tokenizer_output.pkl", "rb") as f:
    tokenizer_output = pickle.load(f)

# Building Chatbot

In [39]:
def chatbot_response(input_text):
    seq = tokenizer_input.texts_to_sequences([input_text])
    padded = pad_sequences(seq, maxlen=max_input_len, padding='post')

    states_value = encoder_model_inf.predict(padded)

    target_seq = np.zeros((1, 1))

    if '<sos>' not in tokenizer_output.word_index:
        tokenizer_output.word_index['<sos>'] = len(tokenizer_output.word_index) + 1

    target_seq[0, 0] = tokenizer_output.word_index['<sos>']

    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        output_tokens, h, c = decoder_model_inf.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_output.index_word.get(sampled_token_index, '')

        if sampled_word == '<eos>' or len(decoded_sentence.split()) > max_output_len:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

            target_seq[0, 0] = sampled_token_index

            states_value = [h, c]

    return decoded_sentence.strip()


In [ ]:
while True:
    input_text = input("Patient: ")
    if input_text.lower() == "exit":
        break
    response = chatbot_response(input_text)
    print("Doctor:", response)


Patient: hi 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/st